# Maximum a Posteriori Parameter Inference

In this notebook, we introduce the Maximum a Posteriori (MAP), which extends Maximum Likelihood Estimation (MLE) by inclusion of a prior $p(\theta)$ into the cost function. To include this prior information, we construct a Bayesian Posterior with Bayesian's Theorem given as,

$$
P(\theta|D) = \frac{P(D|\theta)P(\theta)}{P(D)}
$$

where,  
$~$  
$P(\theta|D)$ represents the posterior and can be read as "the probability of the parameters $(\theta)$ given the data $(D)$",  
$P(D|\theta)$ is the probability of the data given the parameters, commonly called the likelihood,  
$P(\theta)$ represents the probability of the parameters commonly called the prior,  
$P(D)$ is the probability of the data and is commonly called the marginal probability.  

However, as the marginal probability is commonly difficult to compute and represents a normalisation constant, in the case of MAP this term is forgone and the proportional posterior is optmised instead. This is given as,

$$
P(\theta|D) \propto P(D|\theta)P(\theta)
$$

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install and import PyBOP as well as upgrade dependencies. We also fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

To demonstrate the MAP process, we will first need a forward model and data to run parameter inference on. As we are introducing this as a simple example, we will use the PyBaMM forward model with white noise as the reference. This requires defining parameter values and the model itself.

In [ ]:
parameter_values = pybamm.ParameterValues("Chen2020")
model = pybamm.lithium_ion.SPM()

We can now simulate the model using the `pybamm.Simulation` class. For this example, we use the default current function for the "Chen2020" parameter set (5A) to generate the voltage data. As the goal is to investigate the MAP method, we will generate a range of observations from the forward model. 

In [ ]:
n = 6  # Number of time-series trajectories
observations = [
    2**j for j in range(1, n + 1)
]  # Number of observations in each trajectory
values = []
for i in observations:
    t_eval = np.linspace(0, 10, i)
    sim = pybamm.Simulation(model, parameter_values=parameter_values)
    sol = sim.solve(t_eval=t_eval)
    values.append(sol)

print(f"Number of observations in each trajectory: {observations}")

/home/dibyendu/Oxford_Work/PyBOP/newenv/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



Number of observations in each trajectory: [2, 4, 8, 16, 32, 64]


To make the parameter inference more realistic, we add gaussian noise with zero mean to the data. While this doesn't truly represent the challenge of parameter inference with experimental data, this does ensure the cost landscape curvature isn't perfect. For a more realistic representation of experimental data, a different noise function could be used. 

In [ ]:
sigma = 0.005
corrupt_values = values[1]["Voltage [V]"].data + np.random.normal(0, sigma, len(t_eval))

The reference trajectory needs to be included in the optimisation task, which is handed within the `Dataset` class. In this situation, this class is composed of the time, current, and the noisy voltage data; however, if we were performing parameter inference from a different measured signal, such as 'Cell Temperature', that would need to be included.

In [ ]:
dataset = pybop.Dataset(
    {
        "Time [s]": values[1]["Time [s]"].data,
        "Current function [A]": values[1]["Current [A]"].data,
        "Voltage [V]": corrupt_values,
    }
)

## Setting up the problem

Next, we need to select the forward model parameters for inference. The PyBOP parameters class composes as many individual PyBOP parameters as the user wants (whether these parameters can be identified is left out of this example). This class requires the parameter name, which must resolve to a parameter within the `ParameterSet` defined above. Additionally, this class can accept an `initial_value` which will be used by the optimiser, as well as bounds. For this example, we will provide a `prior` to the parameter class, which will be used later by the MAP process.

In [ ]:
parameters = [
    pybop.Parameter(
        "Negative particle radius [m]",
        prior=pybop.Gaussian(4e-6, 1e-6),
        bounds=(1e-6, 1e-5),
    ),
    pybop.Parameter(
        "Positive particle radius [m]",
        prior=pybop.Gaussian(5e-6, 1e-6),
        bounds=(1e-6, 1e-5),
    ),
]

We use a negative Gaussian log-likelihood (NLL) function. Since we have not provided a `sigma` value to the NLL, this will be estimated from the data. `sigma` is the standard deviation of the measurement noise in the dataset.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_values)
    .add_cost(
        pybop.costs.pybamm.NegativeGaussianLogLikelihood("Voltage [V]", "Voltage [V]")
    )
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

## Identifying the Maximum a Posteriori values
Below we identify the parameters using the Maximum a Posterior estimate and the Covariance Matrix Adaptation Evolution Strategy (CMAES).

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=40,
    max_iterations=200,
)
optim = pybop.CMAES(problem, options=options)
results = optim.run()

Next, we can plot the MAP results:

In [ ]:
print("Optimisation results: ", results.x)
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

Optimisation results:  [5.39510998e-06 5.73305869e-06 1.59132199e-02]


### Concluding Thoughts

This notebook illustrates the process of parameter inference with the Maximum a Posteriori method. This process enables encapsulation of prior knowledge into the optimisation process with influence decay as observations of the system increase. This influence decay has been presented above across observations obtained from the set $({2^n \mid n \in \mathbb{N}})$.